<a href="https://colab.research.google.com/github/panchamdesai777/Hackathons/blob/master/Insurance_churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Insurance Churn Prediction**

---

![alt text](https://analyticsindiamag.com/wp-content/uploads/2020/04/Insurance_Churn-scaled.jpg)


### **Description:**

Insurance companies around the world operate in a very competitive environment. With various aspects of data collected from millions of customers, it is painstakingly hard to analyze and understand the reason for a customer’s decision to switch to a different insurance provider.

For an industry where customer acquisition and retention are equally important, and the former being a more expensive process, insurance companies rely on data to understand customer behaviour to prevent retention. Thus knowing whether a customer is possibly going to switch beforehand gives Insurance companies an opportunity to come up with strategies to prevent it from actually happening.

### **Dataset Description :** 

Given are 16 distinguishing factors that can help in understanding the customer churn, your objective as a data scientist is to build a Machine Learning model that can predict whether the insurance company will lose a customer or not using these factors.
You are provided with 16 anonymized factors (feature_0 to feature 15) that influence the churn of customers in the insurance industry

### **Overview of Train Data**

![alt text](https://www.machinehack.com/wp-content/uploads/2020/04/Screenshot-2020-04-22-at-12.04.50-PM-1024x201.png)


### **Overview of Test Data**

![alt text](https://www.machinehack.com/wp-content/uploads/2020/04/Screenshot-2020-04-22-at-12.06.59-PM-1024x212.png)

### **Sample Submission File**

![alt text](https://www.machinehack.com/wp-content/uploads/2020/04/Screenshot-2020-04-22-at-12.08.22-PM.png)

